In [1]:
import json

import numpy as np

import pandas as pd

In [2]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_vaccinazioni_covid = config_data['path_covid_19_france_vaccinations_age_sexe_dep']
file_path_andamento_dei_prezzi_al_consumo = config_data['path_economicref_france_indice_prix_consommation']
file_path_risultati_delle_elezioni_legislative_1_turno = config_data['path_elections_france_legislatives_2022_1er_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_legislative_2_turno = config_data['path_elections_france_legislatives_2022_2nd_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_presidenziali_1_turno = config_data['path_elections_france_presidentielles_2022_1er_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_presidenziali_2_turno = config_data['path_elections_france_presidentielles_2022_2nd_tour_par_bureau_de_vote']
file_path_dati_sulla_disoccupazione = config_data['path_labouref_france_departement_quarter_jobseeker']
file_path_andamento_dei_prezzi_dei_carburanti = config_data['path_prix_des_carburants_j_1_001']

In [4]:
# Carico i dataset in memoria
# TODO:assegnare id dtype per ogni colonna per evitare che pandas faccia 'Dtype Guessing' poichè molto inefficiente 

#df_vaccinazioni_covid = pd.read_csv(file_path_vaccinazioni_covid, sep=';')

#df_risultati_elezioni_legislative_1_turno = pd.read_csv(file_path_risultati_delle_elezioni_legislative_1_turno, sep=';')
#df_risultati_elezioni_legislative_2_turno = pd.read_csv(file_path_risultati_delle_elezioni_legislative_2_turno, sep=';')
#df_risultati_elezioni_presidenziali_1_turno = pd.read_csv(file_path_risultati_delle_elezioni_presidenziali_1_turno, sep=';')
#df_risultati_elezioni_presidenziali_2_turno = pd.read_csv(file_path_risultati_delle_elezioni_presidenziali_2_turno, sep=';')
#df_disoccupazione = pd.read_csv(file_path_dati_sulla_disoccupazione, sep=';')
#df_andamento_prezzi_carburanti = pd.read_csv(file_path_andamento_dei_prezzi_dei_carburanti, sep=';')

C:\Users\terraque\AppData\Local\Temp\ipykernel_3164\564587042.py:6: DtypeWarning: Columns (0,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_risultati_elezioni_legislative_1_turno = pd.read_csv(file_path_risultati_delle_elezioni_legislative_1_turno, sep=';')
C:\Users\terraque\AppData\Local\Temp\ipykernel_3164\564587042.py:8: DtypeWarning: Columns (0,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_risultati_elezioni_presidenziali_1_turno = pd.read_csv(file_path_risultati_delle_elezioni_presidenziali_1_turno, sep=';')
C:\Users\terraque\AppData\Local\Temp\ipykernel_3164\564587042.py:11: DtypeWarning: Columns (20,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_andamento_prezzi_carburanti = pd.read_csv(file_path_andamento_dei_prezzi_dei_carburanti, sep=';')


In [8]:
# Carico e Pulisco dataset "andamento_prezzi_consumo" 

df_andamento_prezzi_consumo = pd.read_csv(file_path_andamento_dei_prezzi_al_consumo, sep=';')
df_andamento_prezzi_consumo.rename(columns={"Période" : "periodo", "Indice" : "suggerimento", "Valeur": "valore","Code" : "codice","Type" : "tipo","Sous-type" : "sottotipo", "Base" : "base", "Population" : "popolazione", "Zone concernée" : "zona_interessata", "Code COICOP" : "codice_COICOP", "Désignation COICOP" : "designazione_COICOP", "Série arrêtée" : "serie_interrotta", "Référence INSEE" : "riferimento_INSEE"}, inplace=True)
df_andamento_prezzi_consumo.head(10)

,periodo,suggerimento,valore,codice,tipo,sottotipo,base,popolazione,zona_interessata,codice_COICOP,designazione_COICOP,serie_interrotta,riferimento_INSEE
0,2017-02,Indice des prix à la consommation - Base 2015 ...,101.67,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France,09.2.2,Instruments de musique et biens durables desti...,Non,1763723
1,2017-02,Indice des prix à la consommation - Base 2015 ...,88.77,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France,09.5.1.4,Services d'édition et livres numériques,Non,1763762
2,2017-02,Indice des prix à la consommation - Base 2015 ...,101.19,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France,11.2.0,Services d'hébergement,Non,1763789
3,2017-02,Indice des prix à la consommation - Base 2015 ...,102.12,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France,11.2.0.3,Autres services d'hébergement,Non,1763792
4,2017-02,Indice des prix à la consommation - Base 2015 ...,101.02,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France,12.7,Autres services n.c.a.,Non,1763828
5,2017-02,Indice des prix à la consommation - Base 2015 ...,102.04,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France métropolitaine,01.1.8.3,Chocolat,Non,1763922
6,2017-02,Indice des prix à la consommation - Base 2015 ...,99.54,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France métropolitaine,01.2,Boissons non alcoolisées,Non,1763932
7,2017-02,Indice des prix à la consommation - Base 2015 ...,87.80,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France métropolitaine,03.2.1,Chaussures diverses,Non,1763973
8,2017-02,Indice des prix à la consommation - Base 2015 ...,101.88,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France métropolitaine,04.4.4,Services divers liés au logement n.c.a.,Non,1763999
9,2017-02,Indice des prix à la consommation - Base 2015 ...,97.90,A,Indice des prix à la consommation,Valeurs mensuelles,Base 2015,Ensemble des ménages,France métropolitaine,05.2.0.2,Linge de lit et accessoires de literie,Non,1764030
